# Import Files

In [7]:
import numpy as np
import pandas as pd
import librosa as lb
import tensorflow as tf
import os
import pathlib

# Import Dataset

In [8]:
if os.path.isdir("audio"):
    audio_dataset = pathlib.Path('audio')
    # Create list of all audio files for tensorflow
    audio_wav = tf.io.gfile.glob(str(audio_dataset) + '/*')
else:
    exit("No audio dataset folder found")

# Audio processing


# Import and Split Dataset

# Results